In [56]:
import sys
import numpy as np
import pandas as pd
import scipy
import copy
import random
import math
from scipy import stats
from scipy.stats import rankdata
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn import metrics, preprocessing
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor, plot_tree
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display
np.random.seed(1)

In [57]:
cols = ['sex', 'rank', 'year', 'degree', 'y_degree', 'salary']
df = pd.read_table('salary.data', names=cols, sep="\t", index_col=False)

**Pre-processing** (categorical to numerical)

In [58]:
 def preprocess(df):
    df.loc[(df['year'] <= 5), 'year'] = 0
    df.loc[(df['year'] > 5) & (df['year'] <= 10), 'year'] = 1
    df.loc[(df['year'] > 10) & (df['year'] <= 15), 'year'] = 2
    df.loc[(df['year'] > 15), 'year'] = 3
    
    df.loc[(df['y_degree'] <= 10), 'y_degree'] = 0
    df.loc[(df['y_degree'] > 10) & (df['y_degree'] <= 20), 'y_degree'] = 1
    df.loc[(df['y_degree'] > 20), 'y_degree'] = 2
    
    df['sex'] = 1 - df['sex']
    
    df['salary'] = df['salary'].apply(lambda x : 1 if x >= 23719 else 0) 
    return df

df = preprocess(df)

y = df['salary']
df = df.drop(columns=['salary'])

X_train, X_test, y_train, y_test = train_test_split(df, y, test_size=0.2, random_state=1)
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

**Protected, privileged**

In [59]:
# protected: 'gender'=0
# privileged: 'gender'=1

# protected: 'age'=0
# privileged: 'age'=1

**Parametric Model**

In [60]:
# size=500
# X_train = X_train[0:size]
# y_train = y_train[0:size]

X_train_orig = copy.deepcopy(X_train)
X_test_orig = copy.deepcopy(X_test)

# Scale data: regularization penalty default: ‘l2’, ‘lbfgs’ solvers support only l2 penalties. 
# Regularization makes the predictor dependent on the scale of the features.
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

clf = LogisticRegression(random_state=0, max_iter=300)

**Compute fairness metric**

In [61]:
def computeFairness(y_pred, X_test, y_test, metric): 
    fairnessMetric = 0
    protected_idx = X_test[X_test['sex']==0].index
    numProtected = len(protected_idx)
    privileged_idx = X_test[X_test['sex']==1].index
    numPrivileged = len(privileged_idx)
        
    p_protected = 0
    for i in range(len(protected_idx)):
        p_protected += y_pred[protected_idx[i]][1]
    p_protected /= len(protected_idx)
    
    p_privileged = 0
    for i in range(len(privileged_idx)):
        p_privileged += y_pred[privileged_idx[i]][1]
    p_privileged /= len(privileged_idx)
    
    # statistical parity difference
    statistical_parity = p_protected - p_privileged
    
#     # equality of opportunity, or 
#     # true positive rate parity
#     # P(Y=1 | Y=1, G=0)- P(Y=1 | Y=1, G=1)
#     true_positive_protected = 0
#     actual_positive_protected = 0
#     for i in range(len(protected_idx)):
#         if (y_test[protected_idx[i]] == 1):
#             actual_positive_protected += 1
# #             if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
#             true_positive_protected += y_pred[protected_idx[i]][1]
#     tpr_protected = true_positive_protected/actual_positive_protected
    
#     true_positive_privileged = 0
#     actual_positive_privileged = 0
#     for i in range(len(privileged_idx)):
#         if (y_test[privileged_idx[i]] == 1):
#             actual_positive_privileged += 1
# #             if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
#             true_positive_privileged += y_pred[privileged_idx[i]][1]
#     tpr_privileged = true_positive_privileged/actual_positive_privileged
    
#     tpr_parity = tpr_protected - tpr_privileged
    
#     # equalized odds or TPR parity + FPR parity
#     # false positive rate parity
    
#     # predictive parity
#     p_o1_y1_s1 = 0
#     p_o1_s1 = 0
#     for i in range(len(protected_idx)):
# #         if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
#         p_o1_s1 += y_pred[protected_idx[i]][1]
#         if (y_test[protected_idx[i]] == 1):
#             p_o1_y1_s1 += y_pred[protected_idx[i]][1]
#     ppv_protected = p_o1_y1_s1/p_o1_s1
    
#     p_o1_y1_s0 = 0
#     p_o1_s0 = 0
#     for i in range(len(privileged_idx)):
# #         if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
#         p_o1_s0 += y_pred[privileged_idx[i]][1]
#         if (y_test[privileged_idx[i]] == 1):
#             p_o1_y1_s0 += y_pred[privileged_idx[i]][1]
#     ppv_privileged = p_o1_y1_s0/p_o1_s0
    
#     predictive_parity = ppv_protected - ppv_privileged
    
    if (metric == 0):
        fairnessMetric = statistical_parity
    elif (metric == 1):
        fairnessMetric = tpr_parity
    elif (metric == 2):
        fairnessMetric = predictive_parity
        
    return fairnessMetric

**Influence of points computed using ground truth**

In [62]:
def ground_truth_influence(X_train, y_train, X_test, X_test_orig, y_test):
    clf.fit(X_train, y_train)
    y_pred = clf.predict_proba(X_test)
    spd_0 = computeFairness(y_pred, X_test_orig, y_test, 0)

    delta_spd = []
    for i in range(len(X_train)):
        X_removed = np.delete(X_train, i, 0)
        y_removed = y_train.drop(index=i, inplace=False)
        clf.fit(X_removed, y_removed)
        y_pred = clf.predict_proba(X_test)
        delta_spd_i = computeFairness(y_pred, X_test_orig, y_test, 0) - spd_0
        delta_spd.append(delta_spd_i)
    
    return delta_spd

**Loss function** (Log loss for logistic regression)

In [63]:
def logistic_loss(y_true, y_pred):
    loss = 0
    for i in range(len(y_true)):
        if (y_pred[i][1] != 0 and y_pred[i][0] != 0):
            loss += - y_true[i] * math.log(y_pred[i][1]) - (1 - y_true[i]) * math.log(y_pred[i][0])
    loss /= len(y_true)
    return loss

**Compute Accuracy** 

In [64]:
from sklearn.metrics import accuracy_score

def computeAccuracy(y_true, y_pred):
    accuracy = 0
    for i in range(len(y_true)):
        idx = y_true[i]
        if (y_pred[i][idx] > y_pred[i][1 - idx]):
            accuracy += 1
#         accuracy += y_pred[i][idx]
    accuracy /= len(y_true)
    return accuracy

**First-order derivative of loss function at z with respect to model parameters**

In [65]:
def del_L_del_theta_i(num_params, y_true, x, y_pred):
#     del_L_del_theta = np.ones((num_params, 1)) * ((1 - y_true) * y_pred[1] - y_true * y_pred[0])
    del_L_del_theta = np.ones((num_params, 1)) * (- y_true + y_pred[1])
    for j in range(1, num_params):
            del_L_del_theta[j] *=  x[j-1]
    return del_L_del_theta

**First-order derivative of $P(y \mid \textbf{x})$ with respect to model parameters**

In [66]:
def del_f_del_theta_i(num_params, x, y_pred):
    del_f_del_theta = np.ones((num_params, 1)) * (y_pred[0] * y_pred[1])
    for j in range(1, num_params):
            del_f_del_theta[j] *=  x[j-1]
    return del_f_del_theta

**Computing $v=\nabla($Statistical parity difference$)$**

In [67]:
# Return v = del(SPD)/del(theta)
def del_spd_del_theta(num_params, X_test_orig, X_test, y_pred):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    numPrivileged = X_test_orig['sex'].sum()
    numProtected = len(X_test_orig) - numPrivileged
    for i in range(len(X_test)):
        del_f_i = del_f_del_theta_i(num_params, X_test[i], y_pred[i])
        if X_test_orig.iloc[i]['sex'] == 1: #privileged
            del_f_privileged = np.add(del_f_privileged, del_f_i)
        elif X_test_orig.iloc[i]['sex'] == 0:
            del_f_protected = np.add(del_f_protected, del_f_i)
    del_f_privileged /= numPrivileged
    del_f_protected /= numProtected
    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Computing $v=\nabla($TPR parity difference$)$**

In [68]:
# Return v = del(TPR_parity)/del(theta)
def del_tpr_parity_del_theta(num_params, X_test_orig, X_test, y_pred, y_test):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    
    protected_idx = X_test_orig[X_test_orig['sex']==0].index
    privileged_idx = X_test_orig[X_test_orig['sex']==1].index

    actual_positive_privileged = 0
    for i in range(len(privileged_idx)):
        if (y_test[privileged_idx[i]] == 1):
            actual_positive_privileged += 1
#             if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
            del_f_i = del_f_del_theta_i(num_params, X_test[privileged_idx[i]], y_pred[privileged_idx[i]])
            del_f_privileged = np.add(del_f_privileged, del_f_i)
    del_f_privileged /= actual_positive_privileged
    
    actual_positive_protected = 0
    for i in range(len(protected_idx)):
        if (y_test[protected_idx[i]] == 1):
            actual_positive_protected += 1
#             if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
            del_f_i = del_f_del_theta_i(num_params, X_test[protected_idx[i]], y_pred[protected_idx[i]])
            del_f_protected = np.add(del_f_protected, del_f_i)
    del_f_protected /= actual_positive_protected

    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Computing $v=\nabla($Predictive parity difference$)$**

In [69]:
# Return v = del(Predictive_parity)/del(theta)
def del_predictive_parity_del_theta(num_params, X_test_orig, X_test, y_pred, y_test):
    del_f_protected = np.zeros((num_params, 1))
    del_f_privileged = np.zeros((num_params, 1))
    
    protected_idx = X_test_orig[X_test_orig['sex']==0].index
    privileged_idx = X_test_orig[X_test_orig['sex']==1].index

    u_dash_protected = np.zeros((num_params, 1))
    v_protected = 0
    v_dash_protected = np.zeros((num_params, 1))
    u_protected = 0
    for i in range(len(protected_idx)):
        del_f_i = del_f_del_theta_i(num_params, X_test[protected_idx[i]], y_pred[protected_idx[i]])
#         if (y_pred[protected_idx[i]][1] > y_pred[protected_idx[i]][0]):
        v_protected += y_pred[protected_idx[i]][1]
        v_dash_protected = np.add(v_dash_protected, del_f_i)
        if (y_test[protected_idx[i]] == 1):
            u_dash_protected = np.add(u_dash_protected, del_f_i)
            u_protected += y_pred[protected_idx[i]][1]
    del_f_protected = (u_dash_protected * v_protected - u_protected * v_dash_protected)/(v_protected * v_protected)
    
    u_dash_privileged = np.zeros((num_params, 1))
    v_privileged = 0
    v_dash_privileged = np.zeros((num_params, 1))
    u_privileged = 0
    for i in range(len(privileged_idx)):
        del_f_i = del_f_del_theta_i(num_params, X_test[privileged_idx[i]], y_pred[privileged_idx[i]])
#         if (y_pred[privileged_idx[i]][1] > y_pred[privileged_idx[i]][0]):
        v_privileged += y_pred[privileged_idx[i]][1]
        v_dash_privileged = np.add(v_dash_privileged, del_f_i)
        if (y_test[privileged_idx[i]] == 1):
            u_dash_privileged = np.add(u_dash_privileged, del_f_i)
            u_privileged += y_pred[privileged_idx[i]][1]
    del_f_privileged = (u_dash_privileged * v_privileged - u_privileged * v_dash_privileged)/(v_privileged * v_privileged)
    
    v = np.subtract(del_f_protected, del_f_privileged)
    return v

**Metrics: Initial state**

In [70]:
clf.fit(X_train, y_train)
num_params = len(clf.coef_.transpose()) + 1 #weights and intercept; params: clf.coef_, clf.intercept_
y_pred_test = clf.predict_proba(X_test)
y_pred_train = clf.predict_proba(X_train)

spd_0 = computeFairness(y_pred_test, X_test_orig, y_test, 0)
print("Initial statistical parity: ", spd_0)

# tpr_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 1)
# print("Initial TPR parity: ", tpr_parity_0)

# predictive_parity_0 = computeFairness(y_pred_test, X_test_orig, y_test, 2)
# print("Initial predictive parity: ", predictive_parity_0)

loss_0 = logistic_loss(y_test, y_pred_test)
print("Initial loss: ", loss_0)

accuracy_0 = computeAccuracy(y_test, y_pred_test)
print("Initial accuracy: ", accuracy_0)

Initial statistical parity:  -0.36341659011731686
Initial loss:  0.23515543948776757
Initial accuracy:  0.8181818181818182


**Pre-compute: del_L_del_theta for each training data point**

In [71]:
del_L_del_theta = []
for i in range(int(len(X_train))):
    del_L_del_theta.insert(i, del_L_del_theta_i(num_params, y_train[i], X_train[i], y_pred_train[i]))

*Select delta fairness function depending on selected metric*

In [72]:
metric = 0
if metric == 0:
    v1 = del_spd_del_theta(num_params, X_test_orig, X_test, y_pred_test)
elif metric == 1:
    v1 = del_tpr_parity_del_theta(num_params, X_test_orig, X_test, y_pred_test, y_test)
elif metric == 2:
    v1 = del_predictive_parity_del_theta(num_params, X_test_orig, X_test, y_pred_test, y_test)

** Update**

In [73]:
def del_L_del_delta_i(num_params, x, y_pred, params, y_true):
    del_L_del_delta = np.ones((num_params - 1, num_params)) * (y_pred[0] * y_pred[1])
    for i in range(num_params - 1):
        for j in range(num_params):
            del_L_del_delta[i][j] *=  params[i]
            if j != 0:
                del_L_del_delta[i][j] *=  x[j - 1]
                if j == i:
                    del_L_del_delta[i][j] += y_pred[1] - y_true
            
    return del_L_del_delta

In [74]:
def repair(idx, numIter, learningRate):
    clf.fit(X_train, y_train)
    y_pred_test = clf.predict_proba(X_test)
    
    X_p = copy.deepcopy(X_train[idx])
    y_p_true = copy.deepcopy(y_train[idx]) 

    del_L_del_delta = np.zeros((num_params - 1, num_params))

    random.seed(0) # seed random number generator
    delta_new = np.zeros((num_params - 1, 1))
    for i in range(len(delta_new)):
        delta_new[i] = random.random()
    
    threshold = 0.05
    delta_old = -1 * np.ones((num_params - 1, 1))
    v1 = del_spd_del_theta(num_params, X_test_orig, X_test, y_pred_test)

    num_iter = 0
    del_L_del_theta_Xp = np.zeros((num_params, 1))
    for i in range(len(idx)):
        del_L_del_theta_Xp = np.subtract(del_L_del_theta_Xp, del_L_del_theta_i(num_params, y_train[idx[i]], X_train[idx[i]], y_pred_train[i]))
    
    obj_old = np.dot(del_L_del_theta_Xp.transpose(), v1)
    obj_new = obj_old - 0.1
    print(obj_old, obj_new)
    
#     while (np.linalg.norm(np.subtract(delta_old, delta_new)) > threshold):
#         print(np.linalg.norm(np.subtract(delta_old, delta_new)))
    while (obj_new < obj_old):
#     while True:
#     while ((num_iter < 100) or (obj_new - obj_old > threshold)) :
        obj_old = obj_new
#         print(num_iter)
        num_iter += 1
        for i in range(len(idx)):
            x = np.zeros((len(X_train[idx[i]]), 1))
            for j in range(len(x)):
                x[j] = X_p[i][j]
            x = np.add(x, delta_new)
            x_ = [x[i][0] for i in range(len(x))]
            y_pred = clf.predict_proba([x_])
            del_L_del_delta_i_ = del_L_del_delta_i(num_params, x, y_pred[0], clf.coef_[0], y_train[idx[i]])/len(idx)
            del_L_del_delta = np.add(del_L_del_delta, del_L_del_delta_i_)
        
        delta_old = delta_new
        delta_new = np.add(delta_new, (learningRate/num_iter) * np.dot(del_L_del_delta, v1))
        
        X_train_perturbed = copy.deepcopy(X_train)
        for i in range(len(idx)):
            for j in range(len(delta_new)):
                X_train_perturbed[idx[i]][j] += delta_new[j]
            
        del_L_del_theta_Xp = np.zeros((num_params, 1))
        for i in range(len(idx)):
            del_L_del_theta_Xp = np.add(del_L_del_theta_Xp, del_L_del_theta_i(num_params, y_train[idx[i]], X_train_perturbed[idx[i]], y_pred_train[idx[i]]))

        obj_new = np.dot(del_L_del_theta_Xp.transpose(), v1)
        print(obj_old, obj_new)
#         if (numIter>=10):
#         if ((num_iter > 10) & (obj_new - obj_old < threshold)):
#         if ((num_iter > 2) & (obj_new < obj_old)):
        if ((obj_new > obj_old)):
#             print((obj_new - obj_old))
            return delta_old
    
    return delta_new

In [161]:
idx = X_train_orig[
    (X_train_orig['rank'] == 2)
    & (X_train_orig['year'] == 2)
#     & (X_train_orig['sex'] == 0)
    & (X_train_orig['degree'] == 1)
    ].index 
# v_pert = repair(idx, numIter, learningRate)
v_pert = repair(idx, 100, 1)

[[-0.18235933]] [[-0.28235933]]
[[-0.28235933]] [[0.22888853]]


In [162]:
df = copy.deepcopy(X_train_orig)
df['salary'] = y_train
df.iloc[idx]

,sex,rank,year,degree,y_degree,salary
12,1,2,2,1,1,1
35,1,2,2,1,1,1


In [155]:
X_test_orig

,sex,rank,year,degree,y_degree
0,1,2,2,0,2
1,1,3,1,1,2
2,1,2,0,1,0
3,1,1,1,0,1
4,0,1,1,0,1
5,1,1,0,1,0
6,0,2,0,0,2
7,1,1,0,1,0
8,1,1,0,1,0
9,1,3,1,0,2


In [158]:
y_pred_test

array([[0.38159269, 0.61840731],
       [0.02896686, 0.97103314],
       [0.4970581 , 0.5029419 ],
       [0.94244923, 0.05755077],
       [0.96682642, 0.03317358],
       [0.9204758 , 0.0795242 ],
       [0.78347191, 0.21652809],
       [0.9204758 , 0.0795242 ],
       [0.9204758 , 0.0795242 ],
       [0.08728766, 0.91271234],
       [0.01616855, 0.98383145]])

In [156]:
%%time
res_a = []
res_b = []
res_c = []
res_d = []
res_e = []

X_train_copy = copy.deepcopy(X_train)
numCols = len(X_train_orig.columns)
for ix in range(len(idx)):
    X_train_pert = np.zeros((len(X_train[idx[ix]]), 1))
    for i in range(len(X_train[idx[ix]])):
        X_train_pert[i] = X_train[idx[ix]][i] + v_pert[i]

    x0 = np.random.rand(1,numCols)
    mins = []
    maxs = []
    numCols = len(X_train[0])
    for i in range(numCols):
        mins.insert(i, min(X_train[:,i]))
        maxs.insert(i, max(X_train[:,i]))

    from scipy.optimize import Bounds, minimize
    bounds = Bounds(mins, maxs)

    f = lambda x: np.linalg.norm(x - X_train_pert)

    x0 = np.random.rand(numCols)
    res = minimize(f, x0, method='trust-constr', 
    #                jac=rosen_der, hess=rosen_hess,
    #                constraints=[linear_constraint, nonlinear_constraint],
                   options={'verbose': 0}, bounds=bounds)
    
    for i in range(len(X_train_copy[idx[ix]])):
        X_train_copy[idx[ix]][i] = res.x[i]
        
#     res_x_inv_transform = sc.inverse_transform(res.x, copy=None)
    res_x_inv_transform = sc.inverse_transform(X_train_copy[idx[ix]], copy=None)
#     res_x_inv_transform = sc.inverse_transform(X_train_copy[idx[ix]], copy=None)
    res_a.insert(ix, res_x_inv_transform[0])
    res_b.insert(ix, res_x_inv_transform[1])
    res_c.insert(ix, res_x_inv_transform[2])
    res_d.insert(ix, res_x_inv_transform[3])
    res_e.insert(ix, res_x_inv_transform[4])
    print(res_x_inv_transform)
#     print(res_x_inv_transform[10], res_x_inv_transform[14], res_x_inv_transform[27], 
#           res_x_inv_transform[0]
#           , res_x_inv_transform[29], res_x_inv_transform[30],
#          res_x_inv_transform[17], res_x_inv_transform[18], res_x_inv_transform[19], res_x_inv_transform[20],
#          res_x_inv_transform[21], res_x_inv_transform[22], res_x_inv_transform[23], res_x_inv_transform[24],
#          res_x_inv_transform[25], res_x_inv_transform[26],
#           '\n')

clf.fit(X_train_copy, y_train)
y_pred_test = clf.predict_proba(X_test)

import statistics
print(statistics.mode(res_a), statistics.mode(res_b), statistics.mode(res_c), statistics.mode(res_d), statistics.mode(res_e))

print(computeFairness(y_pred_test, X_test_orig, y_test, 0)
      /spd_0 - 1
     )
# print(spd_0)


[0.6681155  2.02434211 0.91074517 0.64283421 0.98029857]
0.668115501773171 2.0243421123038954 0.9107451705094671 0.6428342079344276 0.980298572546006
-0.033434925541484195
CPU times: user 369 ms, sys: 36.3 ms, total: 406 ms
Wall time: 226 ms


/Users/dennis/Library/Python/3.8/lib/python/site-packages/scipy/optimize/_hessian_update_strategy.py:182: UserWarning: delta_grad == 0.0. Check if the approximated function is linear. If the function is linear better results can be obtained by defining the Hessian as zero instead of using quasi-Newton approximations.
  warn('delta_grad == 0.0. Check if the approximated '


In [157]:
clf.coef_

array([[0.26228171, 2.09105752, 0.61760523, 0.54211497, 0.1763206 ]])

In [ ]:
Initial: [[ 0.32698744  2.20756884  0.52520941  0.26284814 -0.14408554]]

In [26]:
df = pd.read_table('salary.data', names=cols, sep="\t", index_col=False)
df = preprocess(df)
# df[df['status', 'credit'].groupby(by="credit").sum()

In [27]:
pd.DataFrame(X_train_orig.columns)

,0
0,sex
1,rank
2,year
3,degree
4,y_degree
